In [1]:
!pip install --quiet --upgrade claude-agent-sdk neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.3/325.3 kB 13.1 MB/s eta 0:00:00


The [Claude Agent SDK for Python](https://platform.claude.com/docs/en/agent-sdk/overview) is Anthropic's toolkit for building AI agents. Built on the same agent harness that powers Claude Code, Claude it provides automatic context management, a rich tool ecosystem, fine-grained permissions, and production essentials like error handling and session management.

In this notebook, we'll build a company research agent that queries a Neo4j graph database.

In [2]:
import json

from claude_agent_sdk import (
    ClaudeAgentOptions,
    AssistantMessage,
    TextBlock,
    ToolUseBlock,
    ResultMessage,
    tool,
    create_sdk_mcp_server,
    ClaudeSDKClient,
    AgentDefinition,
    UserMessage,
    ToolResultBlock
)

from typing import Any, Optional

The Claude Agent SDK supports the Anthropic API as well as Amazon Bedrock, Google Vertex AI, and Microsoft Azure as third-party LLM providers. Here we will be using the Anthropic API.

In [3]:
import os
from getpass import getpass

os.environ["ANTHROPIC_API_KEY"] = getpass("Anthropic API key")

Anthropic API key··········


For this example, we'll use the companies database from the Neo4j demo server, which contains organizations, people, investors, and news articles.

In [4]:
os.environ["NEO4J_URI"] = "neo4j+s://demo.neo4jlabs.com"
os.environ["NEO4J_USERNAME"] = "companies"
os.environ["NEO4J_PASSWORD"] = "companies"
os.environ["NEO4J_DATABASE"] = "companies"

First, we define an `invoke_prompt` function to execute queries and display the agent's step-by-step reasoning, tool calls, and results.

In [5]:
async def invoke_prompt(prompt: str, options: dict) -> dict:
    """Send a prompt to Claude and pretty-print the streaming response for debugging."""
    session_id = None
    result_data = {}
    texts = []

    async with ClaudeSDKClient(options=options) as client:
        await client.query(prompt)
        async for message in client.receive_response():
            if hasattr(message, 'subtype') and message.subtype == 'init':
                session_id = message.data.get('session_id')
                print(f"🔑 Session ID: {session_id}")
            elif isinstance(message, UserMessage):
                for block in message.content:
                    if isinstance(block, ToolResultBlock):
                        print(f"\n🤖 Tool Result (tool_use_id: {block.tool_use_id})")
                        print(f"   {'─'*40}")
                        if isinstance(block.content, list):
                            for item in block.content:
                                if isinstance(item, dict) and item.get('type') == 'text':
                                    text = item.get('text', '')
                                    if len(text) > 500:
                                        print(f"   {text[:500]}...")
                                        print(f"   [truncated - {len(text)} chars total]")
                                    else:
                                        print(f"   {text}")
                        else:
                            print(f"   {block.content}")
                        print(f"   {'─'*40}")
            elif isinstance(message, AssistantMessage):
                for block in message.content:
                    if isinstance(block, ToolUseBlock):
                        print(f"\n🔧 Tool: {block.name}")
                        print(f"   ID: {block.id}")
                        print(f"   Args: {json.dumps(block.input, indent=2)}")
                    elif isinstance(block, TextBlock):
                        print(f"\n💬 {block.text}")
                        texts.append(block.text)
            elif isinstance(message, ResultMessage):
                print(f"\n{'='*50}")
                print(f"✅ Completed in {message.duration_ms}ms")
                print(f"   Turns: {message.num_turns}")
                if message.total_cost_usd:
                    print(f"   Cost: ${message.total_cost_usd:.4f}")
                result_data = {
                    "duration_ms": message.duration_ms,
                    "num_turns": message.num_turns,
                    "total_cost_usd": message.total_cost_usd
                }

    return {
        "session_id": session_id,
        "texts": texts,
        "result": result_data
    }

## Neo4j MCP Server

We'll start by using the official [Neo4j MCP Server](https://github.com/neo4j/mcp) to extend the agent with Neo4j tools. This MCP server provides the agent with capabilities to read the graph schema and execute Cypher queries, enabling it to fetch and analyze data directly from the database.

The following code installs the latest version on Google Colab and similar Linux-based systems. For other operating systems, please consult the [official installation documentation](https://neo4j.com/docs/mcp/current/installation/).

In [6]:
import requests

# Get latest release info from GitHub API
release = requests.get("https://api.github.com/repos/neo4j/mcp/releases/latest").json()
version = release["tag_name"]
print(f"Latest version: {version}")

# Download the latest Linux binary
!wget -q https://github.com/neo4j/mcp/releases/download/{version}/neo4j-mcp_Linux_x86_64.tar.gz

# Extract
!tar -xzf neo4j-mcp_Linux_x86_64.tar.gz

# Make executable
!chmod +x neo4j-mcp

# Cleanup
!rm neo4j-mcp_Linux_x86_64.tar.gz

# Move
!mv neo4j-mcp /usr/local/bin/

# Verify installation
!neo4j-mcp -v

Latest version: v1.1.1
neo4j-mcp version: v1.1.1


In [7]:
cypher_mcp_config = {
    "type": "stdio",
    "command": "neo4j-mcp",
    "env": {
        "NEO4J_URI": os.environ["NEO4J_URI"],
        "NEO4J_USERNAME": os.environ["NEO4J_USERNAME"],
        "NEO4J_PASSWORD": os.environ["NEO4J_PASSWORD"],
        "NEO4J_DATABASE": os.environ["NEO4J_DATABASE"],
    },
}


You can add the MCP server config to the agent by passing it to `ClaudeAgentOptions`, along with a `system_prompt` that instructs the agent how to behave. The `allowed_tools` parameter with `"mcp__neo4j__*"` grants the agent access to all tools provided by the Neo4j MCP server.

In [8]:
options = ClaudeAgentOptions(
        system_prompt="You are a helpful assistant with access to a Neo4j graph database containing company data. Use the available tools to query the database and answer questions.",
        mcp_servers={
            "neo4j": cypher_mcp_config
        },
        allowed_tools=["mcp__neo4j__*"]
    )


Let's test it!

In [9]:
prompt = "How many people are in the database?"

results = await invoke_prompt(prompt, options)

🔑 Session ID: 25507299-e4fb-4b08-a7f5-d3457e2aca7e

💬 I'll query the Neo4j database to count how many people are in it.

🔧 Tool: mcp__neo4j__read-cypher
   ID: toolu_01BF7KxskyctMYnDmBQNTNX1
   Args: {
  "query": "MATCH (p:Person) RETURN count(p) as personCount"
}

🤖 Tool Result (tool_use_id: toolu_01BF7KxskyctMYnDmBQNTNX1)
   ────────────────────────────────────────
   [
  {
    "personCount": 8064
  }
]
   ────────────────────────────────────────

💬 There are **8,064 people** in the database.

✅ Completed in 5437ms
   Turns: 2
   Cost: $0.0193


## Session Management

The SDK supports resuming sessions from previous conversation states, enabling continuous development workflows. Each query returns a `session_id` in the initial system message, which can be passed to the `resume` option in `ClaudeAgentOptions` to continue a previous conversation. The SDK automatically handles loading the conversation history and context when you resume a session, allowing Claude to continue exactly where it left off.

In [10]:
options = ClaudeAgentOptions(
        system_prompt="You are a helpful assistant with access to a Neo4j graph database containing company data. Use the available tools to query the database and answer questions.",
        mcp_servers={
            "neo4j": cypher_mcp_config
        },
        allowed_tools=["mcp__neo4j__*"],
        resume=results["session_id"]
    )


We can test it again

In [11]:
prompt = "What did we just discuss?"

results = await invoke_prompt(prompt, options)

🔑 Session ID: 25507299-e4fb-4b08-a7f5-d3457e2aca7e

💬 We just discussed how many people are in the database. I queried the Neo4j database and found that there are **8,064 people** stored in it.

✅ Completed in 2583ms
   Turns: 1
   Cost: $0.0086


## Custom Tools

Beyond using existing MCP servers, you can also implement your own custom tools and add them directly to the agent. This allows you to create specialized functionality tailored to your specific use case.

First, we need to define a Neo4j driver to establish a connection to the database and an `execute_cypher` helper method to run queries.

In [12]:
from neo4j import AsyncGraphDatabase

# Create async driver
driver = AsyncGraphDatabase.driver(os.environ["NEO4J_URI"], auth=(os.environ["NEO4J_USERNAME"], os.environ["NEO4J_PASSWORD"]))


async def execute_cypher(cypher: str, params: dict) -> list[dict]:
    result = await driver.execute_query(cypher, database_=os.environ["NEO4J_DATABASE"], **params)
    return [record.data() for record in result.records]


The first custom tool uses a fulltext index to search for companies by name. The `@tool` decorator defines the tool's name, description, and input schema, which the agent uses to understand when and how to call it.

In [13]:
@tool(
    "find_companies",
    "List of Companies (company_id, name, summary) by fulltext search",
    {"company_name": str}
)
async def find_companies(args):
    query = """
        CALL db.index.fulltext.queryNodes('entity', $search, {limit: 100})
        YIELD node as c, score WHERE c:Organization
        AND NOT EXISTS { (c)<-[:HAS_SUBSIDIARY]-() }
        RETURN c.id as company_id, c.name as name, c.summary as summary LIMIT 25
    """

    try:
        results = await execute_cypher(query, {"search":args["company_name"]})
        return {
            "content": [{"type": "text", "text": json.dumps(results, indent=2)}]
        }
    except Exception as e:
        return {
            "content": [{"type": "text", "text": f"Error: {e}"}],
            "is_error": True
        }

The second tool retrieves investors for a given company, returning their IDs, names, and types (e.g., Organization or Person).

In [14]:
@tool(
    "get_investments",
    "Returns the investments by a company by name. Returns list of investment ids, names and types.",
    {"company": str}
)
async def get_investors(args: dict[str, Any]) -> dict[str, Any]:
    company = args["company"]
    try:
        results = await execute_cypher("""
            MATCH (o:Organization)-[:HAS_INVESTOR]->(i)
            WHERE o.name = $company OR o.id = $company
            RETURN i.id as id, i.name as name, head(labels(i)) as type
        """, {"company": company})
        return {
            "content": [
                {"type": "text", "text": f"Result: {json.dumps(results, indent=2)}"}
            ]
        }
    except Exception as e:
        return {
            "content": [
                {"type": "text", "text": json.dumps({"error": str(e)})}
            ],
            "isError": True
        }

The SDK allows you to bundle custom tools into an in-process MCP server using `create_sdk_mcp_server`, which can then be added to your agent's configuration.

In [15]:
custom_tools_server = create_sdk_mcp_server(
    name="company",
    version="1.0.0",
    tools=[get_investors, find_companies]
)

custom_tools_server_options = ClaudeAgentOptions(
          system_prompt="""You are a helpful assistant with access to a Neo4j graph database containing company data.
You have access to:
- Neo4j Cypher tools for general database queries
- Use investor tool to find investors
- Use find_companies tool to find companies by name in the database

Use these tools to answer questions about companies and their investors.
Make sure to always validate the company name in the database first!""",
          mcp_servers={
              "neo4j": cypher_mcp_config,
              "company-tools": custom_tools_server
          },
          allowed_tools=["mcp__neo4j__*", "mcp__company-tools__*"]
      )

In this case, the agent has access to both the custom `company` MCP server as well as the existing Cypher MCP server.

We can test it!

In [16]:
prompt = "Which companies has Google invested in?"

results = await invoke_prompt(prompt, custom_tools_server_options)

🔑 Session ID: d05ab867-f1c9-4301-9c10-bbcce3138df5

💬 I'll help you find which companies Google has invested in. Let me first verify the company name in the database and then look up their investments.

🔧 Tool: mcp__company-tools__find_companies
   ID: toolu_01Q3j398zS2uah6TTkAEm62X
   Args: {
  "company_name": "Google"
}

🤖 Tool Result (tool_use_id: toolu_01Q3j398zS2uah6TTkAEm62X)
   ────────────────────────────────────────
   [
  {
    "company_id": "EUFq-3WlpNsq0pvfUYWXOEA",
    "name": "Google",
    "summary": "American multinational technology company"
  },
  {
    "company_id": "EtXkRfjAWMtmrXZD_ly9nOQ",
    "name": "Google",
    "summary": "Google's Web mapping service (launched 2005)"
  },
  {
    "company_id": "EUsIfYc_xMfGbcOV6TEkKAQ",
    "name": "Google",
    "summary": null
  },
  {
    "company_id": "Google Drive",
    "name": "Google Drive",
    "summary": ""
  },
  {
    "company_id": "Google Offers",
 ...
   [truncated - 2189 chars total]
   ───────────────────────────

## Subagents

The Claude Agent SDK supports subagents, which are separate agent instances that the main agent can spawn to handle focused subtasks. Subagents enable parallelization by spinning up multiple agents to work on different tasks simultaneously, and help manage context by using their own isolated context windows. They're useful for tasks like code review, research, test execution, and any workflow that benefits from specialized instructions or parallel processing.

We'll implement a news researcher subagent with its own specialized tools. First, we define two custom tools: `company_articles` to fetch the latest 15 articles for a company, and `article` to retrieve full article details including content and sentiment.

In [17]:
@tool(
    "company_articles",
    "Latest 15 articles (article_id, title, date) for a given company",
    {"company_name": str}
)
async def company_articles(args):
    query = """
        MATCH (o:Organization)<-[:MENTIONS]-(a:Article)
        WHERE o.name = $company_name
        RETURN a.id as article_id, a.title as title, toString(a.date) as date
        ORDER BY a.date DESC
        LIMIT 15
    """

    try:
        results = await execute_cypher(query, {"company_name" : args["company_name"]})
        return {
            "content": [{"type": "text", "text": json.dumps(results, indent=2)}]
        }
    except Exception as e:
        return {
            "content": [{"type": "text", "text": f"Error: {e}"}],
            "is_error": True
        }

@tool(
    "article",
    "Single Article details (article_id, author, title, date, sentiment, site, summary, content) by article id",
    {"article_id": str}
)
async def article(args):
    query = """
        MATCH (a:Article)-[:HAS_CHUNK]->(c:Chunk)
        WHERE a.id = $article_id
        WITH a, c ORDER BY id(c) ASC
        WITH a, collect(c.text) as contents
        RETURN a.id as article_id, a.author as author, a.title as title, toString(a.date) as date,
        a.summary as summary, a.siteName as site, a.sentiment as sentiment, apoc.text.join(contents, ' ') as content
    """

    try:
        results = await execute_cypher(query, {"article_id": args["article_id"]})
        return {
            "content": [{"type": "text", "text": json.dumps(results, indent=2)}]
        }
    except Exception as e:
        return {
            "content": [{"type": "text", "text": f"Error: {e}"}],
            "is_error": True
        }

news_tools_server = create_sdk_mcp_server(
    name="news-tools",
    version="1.0.0",
    tools=[company_articles, article]
)

Next, we define the subagent using `AgentDefinition` with a description that tells the main agent when to delegate, a prompt with specific instructions for the subagent's workflow, and `tools` to restrict it to only the news-related MCP tools.

In [18]:
subagents_config = {
        "news-researcher": AgentDefinition(
            description="Expert at finding and investigating company news. Use for news lookups, article retrieval, and sentiment analysis.",
            prompt="""You are a news research specialist with access to a company news database.

You have these tools:
- company_articles: Get the latest 10 articles for a company by company_name
- article: Get full article details by article_id

Workflow:
1. Use 'company_articles' to get recent news for that company name
2. Use 'article' to retrieve full content when deeper analysis is needed. Make sure to use correct article_id from the previous step.

Provide clear summaries of news findings, highlight sentiment trends, and identify key themes.
If the tools do not provide any information, say you don't know!
Only rely on information from the tools. Do not make up information!
""",
            tools=["mcp__news-tools__*"]
        ),
    }

Finally, we pass the subagent configuration to the main agent via the `agents` parameter. The main agent's `allowed_tools` includes `Task` which enables subagent delegation, while the news tools are only exposed to the subagent and not the main agent directly.

In [19]:
subagent_options = ClaudeAgentOptions(
    system_prompt="""You are a helpful assistant with access to a Neo4j graph database containing company data.
You have access to:
- Neo4j Cypher tools for general database queries
- Use investor tool to find investors
- A news-researcher subagent for finding and investigating company news

Use these tools to answer questions about companies and their investors.
Delegate news-related queries to the news-researcher subagent.
Do not hallucinate or make stuff up!
""",
    mcp_servers={
        "neo4j": cypher_mcp_config,
        "company-tools": custom_tools_server,
        "news-tools": news_tools_server
    },
    allowed_tools=["mcp__neo4j__*", "mcp__company-tools__*", "Task"],
    agents=subagents_config
)

Again let's test it.

In [ ]:
prompt = "What is the latest news about Neo4j?"

results = await invoke_prompt(prompt, subagent_options)

## Summary

In this notebook, we built a company research agent using the Claude Agent SDK with Neo4j:

1. **MCP Integration** — Connected to Neo4j using the official Neo4j MCP server for schema reading and Cypher queries
2. **Session Management** — Demonstrated resuming conversations using `session_id`
3. **Custom Tools** — Created specialized tools (`find_companies`, `get_investments`) and bundled them into an in-process MCP server
4. **Subagents** — Delegated news research to a specialized subagent with its own tools and isolated context

The Claude Agent SDK makes it straightforward to combine multiple tool sources, manage context across sessions, and orchestrate complex multi-agent workflows.